# bitflyerの約定データからn秒足のデータを取得

bitflyerの約定データは連番のIDで管理されているので過去の約定データを取得するためにはその日時に対応する約定IDを指定する必要がある。

日時と約定IDの対照表はないので二分探索で求めたい日時の約定IDを探す

In [8]:
import ccxt 
import requests
from datetime import datetime, timedelta
import dateutil.parser
import time
from logging import getLogger,INFO,FileHandler
from pytz import utc

logger = getLogger(__name__)

SYMBOL_BTCFX = 'FX_BTC_JPY'
ITV_SLEEP = 0.001
OHLC_INTERVAL_SEC = 5


# 約定した日時を取得する関数
def get_exec_datetime(d):
    
    
    exec_date = d["exec_date"].replace('T', ' ')[:-1]
    
    
    # 日本時間から標準時に変換
    return dateutil.parser.parse(exec_date) + timedelta(hours=9)


# 約定データを取得する関数
def get_executions(bf, afterId, beforeId, count):
    
    executions = []
    
    while True:
        try:
            
            # 指定したIDからcount分の約定データを取得
            executions = bf.fetch2(path='executions', api='public', method='GET',
                                  params={"product_code": SYMBOL_BTCFX, "after": afterId,
                                         "before": beforeId, "count": count})
            break
        
        # エラーが出た場合は1秒待ってから再度実行
        except Exception as e:
            print("{}: API call error".fortmat(datetime.now()))
            time.sleep(1)
    
    return executions


# 次の約定データの時間を返す関数
def calc_next_date(dt: datetime) -> datetime:
    
    return dt + timedelta(seconds=OHLC_INTERVAL_SEC)


# データがn秒後を満たしているか確認する関数
def is_empty_execution(executions_date: datetime, ohlc_date: datetime) -> bool:
    
    # 次の約定データの時間がn秒後を満たしている場合はTrueを返す
    return ohlc_date + timedelta(seconds=OHLC_INTERVAL_SEC) <= executions_date


# 次の足の時間を確定する関数
def fix_ohlc_date(executions_date: datetime, ohlc_date: datetime) -> datetime:
    
    if not is_empty_execution(executions_date, ohlc_date):
        
        # 時間更新がFalseである場合ohlc dateを更新しない
        return ohlc_date
    
    # 時間更新がTrueの時 ohlc dateを更新する
    ohlc_date = ohlc_date + timedelta(seconds=OHLC_INTERVAL_SEC)
    
    return fix_ohlc_date(executions_date, ohlc_date)

# 二分探索で任意の日時の約定IDを検索
取得できるのは現在から最大31日前まで

In [24]:
# 検索したい時刻
target_dt = "2021/8/30 0:00:00"

# 検索したい時刻を標準時に変換
dt = datetime.strptime(target_dt + "+0900", "%Y/%m/%d %H:%M:%S" + "%z")
target_date = dt.astimezone(utc)

# target_dateと現在時刻の時間差を求める(単位　時間)
hours = int((time.time() - target_date.timestamp()) / 3600) + 1

# 最新の約定IDを取得 一度に取得できるのは最大で500件
params = dict(product_code="FX_BTC_JPY", count=500)
response = requests.get("https://api.bitflyer.com/v1/getexecutions",params=params).json()

# 最新の約定ID
end_id = response[0]["id"]

print(response[0]["exec_date"]+"+0000")

# 二分探索を始める開始ID
# 1時間平均 5万回約定すると仮定
start_id = end_id - hours * 50000


# 開始IDがtarget_dateより過去のものかをチェック
# start IDの約定履歴を取得
params["before"] = start_id + 1
response = requests.get("https://api.bitflyer.com/v1/getexecutions", params=params).json()
print(response[0]["exec_date"]+"+0000")

# start id の約定日時をdatetime(utc)に変換
start_date = datetime.strptime(response[0]["exec_date"] + "+0000", "%Y-%m-%dT%H:%M:%S.%f" + "%z")

# target_dateより過去になるまでstart_idを減らす
while start_date > target_date:
    
    # 5万件 id を差し引いて約定履歴を取得
    start_id -= 50000
    params["before"] = start_id + 1
    response = requests.get("https://api.bitflyer.com/v1/getexecutions", params=params).json()
    print(response[0]["exec_date"]+"+0000")
    
    # start_idの約定日時をdatetime(utc)に変換
    start_date = datetime.strptime(response[0]["exec_date"]+"+0000", "%Y-%m-%dT%H:%M:%S.%f" + "%z")
    


# 二分探索部分

# 探索範囲が　500件以下になるまで絞り込みを行う
while end_id - start_id > 500:
    
    # id間の中央値を取得
    mid_id = int((start_id + end_id)/2)
    
    # 中央値の約定履歴を取得
    params["before"] = mid_id + 1
    response = requests.get("https://api.bitflyer.com/v1/getexecutions", params=params).json()
    
    # 中央値の約定日時をdatetime(utc)に変換
    mid_date = datetime.strptime(response[0]["exec_date"]+ "+0000","%Y-%m-%dT%H:%M:%S.%f" + "%z")
    
    # mid_dateがtarget_dateの左右のどちらかチェック
    if mid_date < target_date:
        
        # target_dateが中央値より右なので start_idをずらす
        start_id = mid_id
    
    else:
        
        # target_idが中央値より左なので end_idをずらす
        end_id = mid_id

        

# 絞り込んだend_idまでの約定履歴を取得
params["before"] = end_id + 1
response = requests.get("https://api.bitflyer.com/v1/getexecutions", params=params).json()

# 約定履歴は新->古の順なので反転する
execs = response[::-1]

# 指定時刻より後の初めのIDを検索
for ex in execs:
    ex_date = datetime.strptime(ex["exec_date"] + "+0000", "%Y-%m-%dT%H:%M:%S.%f" + "%z")
    
    if target_date < ex_date:
        
        # 検索結果を出力
        print("[target id] {}".format(ex["id"]))
        
        break

2021-09-03T20:34:17.03+0000
2021-08-18T01:15:44.337+0000
[target id] 2255223838
